Karolis Vėgėla, 2016061,
Classes ['Balloon', 'Hamster', 'Cannon']
Using ResNet50

In [1]:
import tensorflow as tf
import numpy as np
import os

In [2]:
DATA_DIRECTORY = '.\\validation'
CATEGORIES = ['balloon', 'hamster', 'cannon']
CATEGORY_METRICS = []
threshold = 0.1

In [3]:
for i in CATEGORIES:
    CATEGORY_METRICS.append({
        'label': i,
        'label_index': 0,
        'TP': 0,
        'FP': 0,
        'TN': 0,
        'FN': 0
    })
CATEGORY_METRICS[0]['label_index'] = 417
CATEGORY_METRICS[1]['label_index'] = 333
CATEGORY_METRICS[2]['label_index'] = 471
CATEGORY_METRICS

[{'label': 'balloon', 'label_index': 417, 'TP': 0, 'FP': 0, 'TN': 0, 'FN': 0},
 {'label': 'hamster', 'label_index': 333, 'TP': 0, 'FP': 0, 'TN': 0, 'FN': 0},
 {'label': 'cannon', 'label_index': 471, 'TP': 0, 'FP': 0, 'TN': 0, 'FN': 0}]

In [4]:
# Paths to all the images
balloon_file_paths = [os.path.join(DATA_DIRECTORY + '\\' + CATEGORIES[0], file) for file in os.listdir(DATA_DIRECTORY + '\\' + CATEGORIES[0])]
hamster_file_paths = [os.path.join(DATA_DIRECTORY + '\\' + CATEGORIES[1], file) for file in os.listdir(DATA_DIRECTORY + '\\' + CATEGORIES[1])]
cannon_file_paths = [os.path.join(DATA_DIRECTORY + '\\' + CATEGORIES[2], file) for file in os.listdir(DATA_DIRECTORY + '\\' + CATEGORIES[2])]

all_image_paths = balloon_file_paths + hamster_file_paths + cannon_file_paths

In [5]:
# Labeling the images.
balloon_labels = [417] * len(balloon_file_paths)
hamster_labels = [333] * len(hamster_file_paths)
cannon_labels = [471] * len(cannon_file_paths)
labels = balloon_labels + hamster_labels + cannon_labels

In [6]:
# Data generator
def data_image_generator(all_image_paths, labels):
    for i in range(len(all_image_paths)):
        image = tf.keras.preprocessing.image.load_img(all_image_paths[i], target_size=(224, 224))
        image = tf.keras.preprocessing.image.img_to_array(image)
        yield image, labels[i]

In [7]:
# Creating a dataset
ds_validate = tf.data.Dataset.from_generator(
    generator=data_image_generator, 
    args = (all_image_paths, labels),
    output_types = (tf.float32, tf.int32)
)
ds_validate = ds_validate.cache()
ds_validate = ds_validate.batch(32)
ds_validate = ds_validate.prefetch(tf.data.AUTOTUNE)

In [8]:
# Initializing model and making predictions
model = tf.keras.applications.ResNet50()
predictions = model.predict(ds_validate)

32/32 [==============================] - 55s 2s/step


In [9]:
# Labels of predictions
predicted_labels = [] 
for i in range(len(labels)):
    predicted_labels.append(np.argmax(predictions[i] >= threshold))

In [10]:
predicted_labels = np.array(predicted_labels)
labels = np.array(labels)

In [11]:
def values_calculator(pred_labels, true_labels, index):
    tp = np.sum((pred_labels == index) & (true_labels == index))
    fp = np.sum((pred_labels == index) & (true_labels != index))
    tn = np.sum((pred_labels != index) & (true_labels != index))
    fn = np.sum((pred_labels != index) & (true_labels == index))
    return tp, fp, tn, fn

In [12]:
tp_balloon, fp_balloon, tn_balloon, fn_balloon = values_calculator(predicted_labels, labels, 417)
tp_hamster, fp_hamster, tn_hamster, fn_hamster = values_calculator(predicted_labels, labels,333)
tp_cannon, fp_cannon, tn_cannon, fn_cannon = values_calculator(predicted_labels, labels, 471)

In [13]:
def matrices_calculate(TP, FP, TN, FN):
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    precision = (TP) / (TP + FP)
    recall = (TP) / (TP + FN)
    f1 = (2 * precision * recall) / (precision + recall)
    return accuracy, precision, recall, f1

In [14]:
balloon_acc, balloon_prec, balloon_recall, balloon_f1 = matrices_calculate(tp_balloon, fp_balloon, tn_balloon, fn_balloon)
hamster_acc, hamster_prec, hamster_recall, hamster_f1 = matrices_calculate(tp_hamster, fp_hamster, tn_hamster, fn_hamster)
cannon_acc, cannon_prec, cannon_recall, cannon_f1 = matrices_calculate(tp_cannon, fp_cannon, tn_cannon, fn_cannon)

In [15]:
def print_results(acc, prec, recall, f1, name):
    print(f'{name} - accuracy: {acc}, precision: {prec}, recall: {recall}, f1: {f1}')

In [16]:
print_results(balloon_acc, balloon_prec, balloon_recall, balloon_f1, 'Balloon')
print_results(hamster_acc, hamster_prec, hamster_recall, hamster_f1, 'Hamster')
print_results(cannon_acc, cannon_prec, cannon_recall, cannon_f1, 'Cannon')

Balloon - accuracy: 0.7637637637637638, precision: 1.0, recall: 0.2912912912912913, f1: 0.4511627906976744
Hamster - accuracy: 0.8328328328328328, precision: 1.0, recall: 0.4984984984984985, f1: 0.6653306613226452
Cannon - accuracy: 0.8348348348348348, precision: 1.0, recall: 0.5045045045045045, f1: 0.6706586826347305
